# RULES :

Step 1 to 5 already done. Now, we do step 6 and 7

1) Replace FLT3_ITD by new column ITD where we have set up the rules.

2) Additions : sum (p and q) and return 1 if sum >=1 for each chromosome . ex : add_10 = add_10p+add_10q

3) Deletions : exactly the same . ex: del_10 = del_10p +del_10q

4) Keep only new columns additions and deletions with frequency >=2%

5) Translocations : Keep all translocations that appear at least two times. For translocations that appear only once : create a new column "other transloc" to sum all those translocation and return 1 when value is >=1

6) Inversions : There is only one inversion with frequency greater than 2% (inv  16). we will add inv3 because it creates its own cluster with SF3B1. (We will not do this in this notebook).

7) Replace complex columns by : 1 if sumn of aneuploidies (any additions and any deletions is >=3)

In [9]:
library(dplyr)

# 6 Add inversion 16 and inversion 3

In [19]:
df_final <- read.table("../../../data/updated_dataset/modif_final.csv",sep = ',' , header = T)
rownames(df_final)<- df_final$data_pd
df_final <- df_final[,-1:-3]
df_master <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_master) <- df_master$data_pd
df_final <- merge(df_final,df_master[c('inv_3','inv_16')],by=0)
rownames(df_final) <- df_final$Row.names
df_final <- df_final[,-1]

# 7 Set up the new column complex 

In [20]:

df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
tmp <- df_initial

tmp <- tmp[,c(124:169,171:210,317:365,367:475,477)]
for (i in 1:22){
    tmp[,paste("add_",as.character(i),sep="")] <- tmp[,paste("add_","p",sep=as.character(i))]+tmp[,paste("add_","q",sep=as.character(i))]+tmp[,paste("plus",as.character(i),sep="")]
    tmp[,paste("add_",as.character(i),sep="")][tmp[,paste("add_",as.character(i),sep="")]>=2] <-1
} 
tmp$add_x <- tmp$add_xp + tmp$add_xq + tmp$plusx
tmp$add_y <- tmp$plusy
###
# deletions
###
for (i in c(c(1,2,3),5:13,15:19)){
    tmp[,paste("del_",as.character(i),sep="")] <- tmp[,paste("del_","p",sep=as.character(i))]+tmp[,paste("del_","q",sep=as.character(i))]+tmp[,paste("minus",as.character(i),sep="")]
    tmp[,paste("del_",as.character(i),sep="")][tmp[,paste("del_",as.character(i),sep="")]>=2] <-1  
}
for (i in c(c(4,14,20,21,22),"x")){
    tmp[,paste("del_",as.character(i),sep="")]<- tmp[,paste("del_","q",sep=as.character(i))]+tmp[,paste("minus",as.character(i),sep="")]
}     
tmp$del_y <- tmp$minusy
tmp$sum <- rowSums(tmp[,246:293],na.rm=T)  ## to sum only for add and del 
tmp$new_complex <- ifelse(tmp$sum>=3,1,0)

# HDP Dataframe

# 1) Without U2AF1 Separation

We call it "aml_data genes.tsv" in AML_Paper Project

In [21]:
df_final <- merge(df_final,tmp['new_complex'],by=0)
rownames(df_final) <- df_final$Row.names 
df_final <- df_final[-1]
df_final$complex <- df_final$new_complex
df_final$new_complex <- NULL

In [ ]:
#write.table(df_final,"modif_final.tsv")

# 2) With U2AF1 Separation

In [60]:
df_hotspots <- read.table("../../../data/initial_dataset/1013.mut.csv",sep=',', header = T)
df_hotspots <- df_hotspots[df_hotspots$gene=="U2AF1",c("data_pd","gene","protein")]
df_hotspots$U2AF1_p.S34 <- ifelse(grepl( "34",df_hotspots$protein) | grepl( "35",df_hotspots$protein),1,0)
df_hotspots$U2AF1_p.Q157 <- ifelse(grepl( "157",df_hotspots$protein) | grepl( "156",df_hotspots$protein),1,0)
df_hotspots <- unique(df_hotspots,by="data_pd")

newd <-  df_hotspots %>% group_by(data_pd) %>% filter(n()>1)

tmp1 <- data.frame(data_pd=unique(newd$data_pd),gene=rep("U2AF1",length(unique(newd$data_pd))),protein=rep("p",length(unique(newd$data_pd))),U2AF1_p.S34=rep(0,length(unique(newd$data_pd))),U2AF1_p.Q157=rep(0,length(unique(newd$data_pd))))
for (element in unique(newd$data_pd)){    
    tmp1[tmp1$data_pd==element,]$U2AF1_p.S34 <- colSums(newd[newd$data_pd==element,"U2AF1_p.S34"])
    tmp1[tmp1$data_pd==element,]$U2AF1_p.Q157 <- colSums(newd[newd$data_pd==element,"U2AF1_p.Q157"])
}
df_hotspots <- rbind(df_hotspots[!is.element(df_hotspots$data_pd,unique(newd$data_pd)),c("data_pd","U2AF1_p.S34","U2AF1_p.Q157","protein")],tmp1[,c("data_pd","U2AF1_p.S34","U2AF1_p.Q157","protein")])
rownames(df_hotspots)<- df_hotspots$data_pd
df_u2af1 <- df_hotspots[,c("U2AF1_p.S34","U2AF1_p.Q157")]
tmp_u2af1 <- merge(df_final,df_u2af1,by=0,all.x=T)
rownames(tmp_u2af1) <- tmp_u2af1$Row.names
tmp_u2af1 <- tmp_u2af1[,c(2:82,155,156,83:154)]
tmp_u2af1[is.na(tmp_u2af1)] <- 0

In [62]:
# write.table(tmp_u2af1,"modif_final_u2af1_sep.tsv")